In [1]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 

june_results = session.query(Measurement.tobs).\
 filter(extract("month", Measurement.date) == 6).all()



In [5]:
#  3. Convert the June temperatures to a list.

june_list = list(np.ravel(june_results))

In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 

june_df = pd.DataFrame(june_list, columns = ["June Temps"])

In [7]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.

june_df.describe()

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.

december_results = session.query(Measurement.tobs).\
 filter(extract("month", Measurement.date) == 12).all()

In [9]:
# 7. Convert the December temperatures to a list.

december_list = list(np.ravel(december_results))

In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 

december_df = pd.DataFrame(december_list, columns = ["December Temps"])

In [11]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.

december_df.describe()

,December Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## D3: Additional Queries

In [12]:
inspector = inspect(engine)

In [16]:
# Determine Column Names in Measurement Table

measure_columns = inspector.get_columns("Measurement")

for columns in measure_columns:
    print(columns["name"], columns["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [18]:
# Determine the Column Names in Stations Table

station_columns = inspector.get_columns("Station")

for columns in station_columns:
    print(columns["name"], columns["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [90]:
join_stmt = session.query(Measurement.station, Station.elevation, func.avg(Measurement.tobs)).\
 join(Station, Measurement.id == Station.id).group_by(Measurement.id).all()

temp_by_station = pd.DataFrame(join_stmt, columns = ["Station", "Elevation", "Avg_Temps"])

temp_by_station

,Station,Elevation,Avg_Temps
0,USC00519397,3.0,65.0
1,USC00519397,14.6,63.0
2,USC00519397,7.0,74.0
3,USC00519397,11.9,76.0
4,USC00519397,306.6,73.0
5,USC00519397,19.5,70.0
6,USC00519397,32.9,64.0
7,USC00519397,0.9,68.0
8,USC00519397,152.4,73.0


In [91]:
# Query to provide summary statistics regarding precipitation for June

june_prcp_results = session.query(Measurement.prcp).\
 filter(extract("month", Measurement.date) == 6).all()

june_prcp_list = list(np.ravel(june_prcp_results))

june_prcp_df = pd.DataFrame(june_prcp_list, columns = ["June Prcp"])

june_prcp_df.describe()

,June Prcp
count,1574.000000
mean,0.136360
std,0.335731
min,0.000000
25%,0.000000
50%,0.020000
75%,0.120000
max,4.430000


In [92]:
# Query to provide summary statistics regarding precipitation for June

dec_prcp_results = session.query(Measurement.prcp).\
 filter(extract("month", Measurement.date) == 12).all()

dec_prcp_list = list(np.ravel(dec_prcp_results))

dec_prcp_df = pd.DataFrame(dec_prcp_list, columns = ["December Prcp"])

dec_prcp_df.describe()

,December Prcp
count,1405.000000
mean,0.216819
std,0.541399
min,0.000000
25%,0.000000
50%,0.030000
75%,0.150000
max,6.420000
